In [2]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
import re
from langdetect import detect
from wordcloud import WordCloud


import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity

from nltk.tag import pos_tag

from nltk.stem.lancaster import LancasterStemmer

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.decomposition import NMF, TruncatedSVD, LatentDirichletAllocation

In [3]:
reviews = pd.read_csv('2018_2021_SF_LA_reviews.csv')
reviews.info()

/Users/stay_one/opt/anaconda3/envs/metis/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1051856 entries, 0 to 1051855
Data columns (total 8 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   listing_id     1051856 non-null  object 
 1   id             1051852 non-null  float64
 2   date           1051852 non-null  object 
 3   reviewer_id    1051850 non-null  float64
 4   reviewer_name  1051850 non-null  object 
 5   comments       1051850 non-null  object 
 6   year           1051848 non-null  float64
 7   city           1051848 non-null  object 
dtypes: float64(3), object(5)
memory usage: 64.2+ MB


In [26]:
reviews.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments,year,city
0,958,225199777.0,2018-01-05,30434241.0,Brandy,Holly's home is really beautiful. Other revie...,2018.0,SF
1,958,237950593.0,2018-02-24,20166310.0,Fernando,El apartamento esta exactamente en las condici...,2018.0,SF
2,958,238773143.0,2018-02-26,2504274.0,Elyse,Perfect location and a lovely home! Holly and ...,2018.0,SF
3,958,239949486.0,2018-03-03,60498611.0,Nik,Holly was fantastic! Amazing location! Holly r...,2018.0,SF
4,958,240918832.0,2018-03-06,5852641.0,Louise,Awesome place. Great host. Excellent location....,2018.0,SF


In [4]:
reviews.dropna(inplace=True)
reviews.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1051848 entries, 0 to 1051855
Data columns (total 8 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   listing_id     1051848 non-null  object 
 1   id             1051848 non-null  float64
 2   date           1051848 non-null  object 
 3   reviewer_id    1051848 non-null  float64
 4   reviewer_name  1051848 non-null  object 
 5   comments       1051848 non-null  object 
 6   year           1051848 non-null  float64
 7   city           1051848 non-null  object 
dtypes: float64(3), object(5)
memory usage: 72.2+ MB


In [13]:
df = reviews[['city', 'year', 'comments']]
df.head()

,city,year,comments
0,SF,2018.0,Holly's home is really beautiful. Other revie...
1,SF,2018.0,El apartamento esta exactamente en las condici...
2,SF,2018.0,Perfect location and a lovely home! Holly and ...
3,SF,2018.0,Holly was fantastic! Amazing location! Holly r...
4,SF,2018.0,Awesome place. Great host. Excellent location....


In [14]:
df['reviews_str'] = df[['comments']].astype('string')

/var/folders/71/xlbnyb5s341dqzxgjd0pzzqw0000gn/T/ipykernel_2324/1012469350.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['reviews_str'] = df[['comments']].astype('string')


In [11]:
df.head()

,city,year,comments,reviews_str
0,SF,2018.0,Holly's home is really beautiful. Other revie...,Holly's home is really beautiful. Other revie...
1,SF,2018.0,El apartamento esta exactamente en las condici...,El apartamento esta exactamente en las condici...
2,SF,2018.0,Perfect location and a lovely home! Holly and ...,Perfect location and a lovely home! Holly and ...
3,SF,2018.0,Holly was fantastic! Amazing location! Holly r...,Holly was fantastic! Amazing location! Holly r...
4,SF,2018.0,Awesome place. Great host. Excellent location....,Awesome place. Great host. Excellent location....


In [29]:
df[['comments']].astype('string')

,comments
0,Holly's home is really beautiful. Other revie...
1,El apartamento esta exactamente en las condici...
2,Perfect location and a lovely home! Holly and ...
3,Holly was fantastic! Amazing location! Holly r...
4,Awesome place. Great host. Excellent location....
...,...
1051851,it's right next to the mAll
1051852,Great location and place to stay
1051853,Tina’s place was extremely clean and spacious....
1051854,Meifang is so kind. Communication was great. E...


In [15]:
#Pre-Processing
df["reviews_str"]= df["reviews_str"].apply(lambda x:re.sub("[^A-Za-z]", " ", x.strip()))

## Lower Case 
#df['reviews_str'] = df['reviews_str'].apply(lambda x: " ".join(x.lower() for x in x.split()))

## Numbers Remove
df['reviews_str'] = df['reviews_str'].str.replace('[\d]', '')

## Puncations Remove
#df['reviews_str'] = df['reviews_str'].str.replace('[^\w\s]','')
df

/var/folders/71/xlbnyb5s341dqzxgjd0pzzqw0000gn/T/ipykernel_2324/3259802042.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["reviews_str"]= df["reviews_str"].apply(lambda x:re.sub("[^A-Za-z]", " ", x.strip()))
/var/folders/71/xlbnyb5s341dqzxgjd0pzzqw0000gn/T/ipykernel_2324/3259802042.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  df['reviews_str'] = df['reviews_str'].str.replace('[\d]', '')
/var/folders/71/xlbnyb5s341dqzxgjd0pzzqw0000gn/T/ipykernel_2324/3259802042.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-

,city,year,comments,reviews_str
0,SF,2018.0,Holly's home is really beautiful. Other revie...,Holly s home is really beautiful Other revie...
1,SF,2018.0,El apartamento esta exactamente en las condici...,El apartamento esta exactamente en las condici...
2,SF,2018.0,Perfect location and a lovely home! Holly and ...,Perfect location and a lovely home Holly and ...
3,SF,2018.0,Holly was fantastic! Amazing location! Holly r...,Holly was fantastic Amazing location Holly r...
4,SF,2018.0,Awesome place. Great host. Excellent location....,Awesome place Great host Excellent location ...
...,...,...,...,...
1051851,LA,2021.0,it's right next to the mAll,it s right next to the mAll
1051852,LA,2021.0,Great location and place to stay,Great location and place to stay
1051853,LA,2021.0,Tina’s place was extremely clean and spacious....,Tina s place was extremely clean and spacious ...
1051854,LA,2021.0,Meifang is so kind. Communication was great. E...,Meifang is so kind Communication was great E...


In [19]:
import spacy
#df['spacy_doc'] = list(nlp.pipe(df.comments))


nlp = spacy.load('en_core_web_sm')

docs = list(nlp.pipe(df.reviews_str))
df['spacy_doc'] = docs

df.to_csv('spacyALL_2018_2021_SF_LA_reviews.csv', index=False)

In [43]:
type(docs)

list

In [ ]:
docs.to

In [25]:
sf = df [df['city'] == 'SF']
sf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 180175 entries, 0 to 180177
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   city         180175 non-null  object 
 1   year         180175 non-null  float64
 2   comments     180175 non-null  object 
 3   reviews_str  180175 non-null  object 
 4   compound     180175 non-null  float64
 5   pos          180175 non-null  float64
 6   neg          180175 non-null  float64
 7   spacy_doc    180175 non-null  object 
dtypes: float64(4), object(4)
memory usage: 12.4+ MB


In [33]:
sf['sentiment'] = np.where(sf['compound'] >= .7, 'pos', (np.where(sf['compound'] < .25, 'neg', 'neutral')))
sf.head()

/var/folders/71/xlbnyb5s341dqzxgjd0pzzqw0000gn/T/ipykernel_2324/3617355054.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sf['sentiment'] = np.where(sf['compound'] >= .7, 'pos', (np.where(sf['compound'] < .25, 'neg', 'neutral')))


,city,year,comments,reviews_str,compound,pos,neg,spacy_doc,compound_score,sentiment
0,SF,2018.0,Holly's home is really beautiful. Other revie...,Holly s home is really beautiful Other revie...,0.9803,0.223,0.014,"(Holly, s, home, is, really, beautiful, , Ot...",0.9803,pos
1,SF,2018.0,El apartamento esta exactamente en las condici...,El apartamento esta exactamente en las condici...,-0.5574,0.000,0.040,"(El, apartamento, esta, exactamente, en, las, ...",-0.5574,neg
2,SF,2018.0,Perfect location and a lovely home! Holly and ...,Perfect location and a lovely home Holly and ...,0.9709,0.455,0.000,"(Perfect, location, and, a, lovely, home, , H...",0.9709,pos
3,SF,2018.0,Holly was fantastic! Amazing location! Holly r...,Holly was fantastic Amazing location Holly r...,0.8126,0.363,0.000,"(Holly, was, fantastic, , Amazing, location, ...",0.8126,pos
4,SF,2018.0,Awesome place. Great host. Excellent location....,Awesome place Great host Excellent location ...,0.9800,0.530,0.000,"(Awesome, place, , Great, host, , Excellent,...",0.9800,pos


In [37]:
positive_sf = sf [ sf['sentiment'] == 'pos']
positive_sf.shape

(144107, 10)

In [41]:
positive_sf.to_pickle('positive_sf.pkl')

In [38]:
negative_sf = sf [ sf['sentiment'] == 'neg']
negative_sf.shape

(11404, 10)

In [39]:
negative_sf.to_pickle('negative_sf.pkl')

In [26]:
la = df [df['city'] == 'LA']
la.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 871673 entries, 180178 to 1051855
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   city         871673 non-null  object 
 1   year         871673 non-null  float64
 2   comments     871673 non-null  object 
 3   reviews_str  871673 non-null  object 
 4   compound     871673 non-null  float64
 5   pos          871673 non-null  float64
 6   neg          871673 non-null  float64
 7   spacy_doc    871673 non-null  object 
dtypes: float64(4), object(4)
memory usage: 59.9+ MB


In [34]:
la['sentiment'] = np.where(la['compound'] >= .7, 'pos', (np.where(la['compound'] < .25, 'neg', 'neutral')))
la.head()

/var/folders/71/xlbnyb5s341dqzxgjd0pzzqw0000gn/T/ipykernel_2324/2865804181.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  la['sentiment'] = np.where(la['compound'] >= .7, 'pos', (np.where(la['compound'] < .25, 'neg', 'neutral')))


,city,year,comments,reviews_str,compound,pos,neg,spacy_doc,sentiment
180178,LA,2018.0,"Chas house feels like home, it is clean, it ha...",Chas house feels like home it is clean it ha...,0.9631,0.365,0.0,"(Chas, house, feels, like, home, , it, is, cl...",pos
180179,LA,2018.0,1. Clean 2. Host is very friendly 3. Reasonabl...,Clean Host is very friendly Reasonabl...,0.9637,0.396,0.0,"( , Clean, , Host, is, very, friendly, ...",pos
180180,LA,2018.0,Wow; Charles was a delight to meet a very warm...,Wow Charles was a delight to meet a very warm...,0.9863,0.411,0.0,"(Wow, , Charles, was, a, delight, to, meet, a...",pos
180181,LA,2018.0,Chas. has a great space ! Loved how clean ever...,Chas has a great space Loved how clean ever...,0.9246,0.427,0.0,"(Chas, , has, a, great, space, , Loved, how...",pos
180182,LA,2018.0,"Chas is a great host, very friendly and welcom...",Chas is a great host very friendly and welcom...,0.9879,0.287,0.0,"(Chas, is, a, great, host, , very, friendly, ...",pos


In [35]:
positive_la = la [ la['sentiment'] == 'pos']
positive_la.shape

(671770, 9)

In [42]:
positive_la.to_pickle('positive_la.pkl')

In [36]:
negative_la = la [ la['sentiment'] == 'neg']
negative_la.shape

(59695, 9)

In [40]:
negative_la.to_pickle('negative_la.pkl')

In [20]:
df.to_pickle('spacyAll')

In [ ]:
# save dataframe to pickle file
df.to_pickle('spacyAll2018_2021_df.pkl')



In [23]:
# read pickle file as dataframe
pickle_df = pd.read_pickle('spacyAll')
pickle_df.head()

,city,year,comments,reviews_str,compound,pos,neg,spacy_doc
0,SF,2018.0,Holly's home is really beautiful. Other revie...,Holly s home is really beautiful Other revie...,0.9803,0.223,0.014,"(Holly, s, home, is, really, beautiful, , Ot..."
1,SF,2018.0,El apartamento esta exactamente en las condici...,El apartamento esta exactamente en las condici...,-0.5574,0.000,0.040,"(El, apartamento, esta, exactamente, en, las, ..."
2,SF,2018.0,Perfect location and a lovely home! Holly and ...,Perfect location and a lovely home Holly and ...,0.9709,0.455,0.000,"(Perfect, location, and, a, lovely, home, , H..."
3,SF,2018.0,Holly was fantastic! Amazing location! Holly r...,Holly was fantastic Amazing location Holly r...,0.8126,0.363,0.000,"(Holly, was, fantastic, , Amazing, location, ..."
4,SF,2018.0,Awesome place. Great host. Excellent location....,Awesome place Great host Excellent location ...,0.9800,0.530,0.000,"(Awesome, place, , Great, host, , Excellent,..."


In [24]:
pickle_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1051848 entries, 0 to 1051855
Data columns (total 8 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   city         1051848 non-null  object 
 1   year         1051848 non-null  float64
 2   comments     1051848 non-null  object 
 3   reviews_str  1051848 non-null  object 
 4   compound     1051848 non-null  float64
 5   pos          1051848 non-null  float64
 6   neg          1051848 non-null  float64
 7   spacy_doc    1051848 non-null  object 
dtypes: float64(4), object(4)
memory usage: 72.2+ MB


In [16]:
df.to_csv('2manualClean_2018_2021_SF_LA_reviews.csv', index=False)

In [17]:
analyzer = SentimentIntensityAnalyzer()

df['compound'] = df.reviews_str.map(analyzer.polarity_scores).map(lambda x: x.get('compound'))
df['pos'] = df.reviews_str.map(analyzer.polarity_scores).map(lambda x: x.get('pos'))
df['neg'] = df.reviews_str.map(analyzer.polarity_scores).map(lambda x: x.get('neg'))


/var/folders/71/xlbnyb5s341dqzxgjd0pzzqw0000gn/T/ipykernel_2324/925784612.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['compound'] = df.reviews_str.map(analyzer.polarity_scores).map(lambda x: x.get('compound'))
/var/folders/71/xlbnyb5s341dqzxgjd0pzzqw0000gn/T/ipykernel_2324/925784612.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['pos'] = df.reviews_str.map(analyzer.polarity_scores).map(lambda x: x.get('pos'))
/var/folders/71/xlbnyb5s341dqzxgjd0pzzqw0000gn/T/ipykernel_2324/925784612.py:5: 

In [18]:
df.to_csv('2vader_scores_2018_2021_SF_LA_reviews.csv', index=False)

In [35]:
df.compound.mean()

0.7813183416235043

In [57]:
df.pos.mean()

0.3536

In [56]:
df.neg.mean()

0.0024000000000000002

In [ ]:
df.compound.mean()

df.pos.mean()

df.neg.mean()

In [ ]:
np.where(consumption_energy > 400, 'high', 
         (np.where(consumption_energy < 200, 'low', 'medium')))

In [ ]:
# above 50% reviews to positive, the rest to negative
df['sentiment2'] = np.where(df['scores'] >= .3, 'positive', 'negative')

In [36]:
df['sentiment'] = np.where(df['compound'] >= .7, 'pos', (np.where(df['compound'] < .25, 'neg', 'neutral')))
df.head()

/var/folders/71/xlbnyb5s341dqzxgjd0pzzqw0000gn/T/ipykernel_1267/3270410805.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sentiment'] = np.where(df['compound'] >= .7, 'pos', (np.where(df['compound'] < .25, 'neg', 'neutral')))


,city,year,comments,compound,pos,neg,sentiment
0,SF,2018.0,holly s home is really beautiful other reviewe...,0.9803,0.223,0.014,pos
1,SF,2018.0,el apartamento esta exactamente en las condici...,-0.5574,0.000,0.040,neg
2,SF,2018.0,perfect location and a lovely home holly and h...,0.9709,0.455,0.000,pos
3,SF,2018.0,holly was fantastic amazing location holly rep...,0.8126,0.363,0.000,pos
4,SF,2018.0,awesome place great host excellent location su...,0.9800,0.530,0.000,pos


In [37]:
df.to_csv('sentiment_2018_2021_SF_LA_reviews.csv', index=False)

In [39]:
df.sentiment.value_counts(normalize=True)

pos        0.775333
neutral    0.157022
neg        0.067646
Name: sentiment, dtype: float64

In [51]:
positive = df [ df['sentiment'] == 'pos']
print(positive.shape)
positive.head()

(815532, 7)


,city,year,comments,compound,pos,neg,sentiment
0,SF,2018.0,holly s home is really beautiful other reviewe...,0.9803,0.223,0.014,pos
2,SF,2018.0,perfect location and a lovely home holly and h...,0.9709,0.455,0.000,pos
3,SF,2018.0,holly was fantastic amazing location holly rep...,0.8126,0.363,0.000,pos
4,SF,2018.0,awesome place great host excellent location su...,0.9800,0.530,0.000,pos
5,SF,2018.0,todo perfecto la casa es hermosa br la ubicaci...,0.9169,0.133,0.000,pos


In [55]:
sf_pos = positive [ positive['city'] == 'SF']
print(sf_pos.shape)

la_pos = positive [ positive['city'] == 'LA']
print(la_pos.shape)

(144045, 7)
(671487, 7)


In [53]:
negative = df [ df['sentiment'] == 'neg']
print(negative.shape)
negative.head()

(71153, 7)


,city,year,comments,compound,pos,neg,sentiment
1,SF,2018.0,el apartamento esta exactamente en las condici...,-0.5574,0.0,0.040,neg
12,SF,2018.0,br br br br br br,0.0000,0.0,0.000,neg
62,SF,2019.0,la ubicaci n es estupenda viajando con un pequ...,-0.2960,0.0,0.019,neg
67,SF,2019.0,on a very quiet residential street but close t...,0.0000,0.0,0.000,neg
73,SF,2019.0,este apartamento es muy recomendable tiene de ...,-0.2960,0.0,0.017,neg


In [56]:
sf_neg = negative [ negative['city'] == 'SF']
print(sf_neg.shape)

la_neg = negative [ negative['city'] == 'LA']
print(la_neg.shape)

(11412, 7)
(59741, 7)


In [59]:
sf_pos.to_csv('sf_positive.csv', index=False)

In [57]:
sf_neg.to_csv('sf_negative.csv', index=False)

In [60]:
la_neg.to_csv('la_positive.csv', index=False)

In [58]:
la_neg.to_csv('la_negative.csv', index=False)